In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip3 install category_encoders
import category_encoders as ce

     |████████████████████████████████| 82 kB 317 kB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# import seaborn as sns

from sklearn.model_selection import KFold

In [5]:
!cp /content/drive/"My Drive"/e3.csv /content/

In [6]:
csvdata = pd.read_csv('e3.csv')

In [7]:

del csvdata['attackType']
del csvdata['attackID']
del csvdata['attackDescription']
del csvdata['Flows']
del csvdata['Tos']
csvdata

,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Flags,class,Bytes
0,0.000,TCP,26913_25,43618,EXT_SERVER,23.0,1,....S.,suspicious,46
1,0.000,TCP,EXT_SERVER,23,26913_25,43618.0,1,.A.R..,suspicious,40
2,0.000,TCP,26914_116,1817,EXT_SERVER,23.0,1,....S.,suspicious,46
3,0.000,TCP,EXT_SERVER,23,26914_116,1817.0,1,.A.R..,suspicious,40
4,0.001,TCP,26915_202,57272,EXT_SERVER,1433.0,1,....S.,suspicious,46
...,...,...,...,...,...,...,...,...,...,...
153021,67601.294,TCP,OPENSTACK_NET,49493,EXT_SERVER,8082.0,160681,.APRS.,normal,12200000
153022,67601.294,TCP,EXT_SERVER,8082,OPENSTACK_NET,49494.0,172810,.AP.S.,normal,465700000
153023,67601.294,TCP,OPENSTACK_NET,49494,EXT_SERVER,8082.0,170412,.APRS.,normal,28500000
153024,67601.103,TCP,EXT_SERVER,8082,OPENSTACK_NET,49495.0,148533,.AP.S.,normal,394900000


In [8]:
multipliers = {'K':1000, 'M':1000000, 'B':1000000000}

def string_to_int(string):
  if string.find('M') == -1:
      return int(string)
  mult = multipliers[string[-1]] # look up suffix to get multiplier
  # convert number to float, multiply by multiplier, then make int
  return int(float(string[:-1]) * mult)

bytes_new = list(map(string_to_int, csvdata['Bytes'].astype('str') ))
print("len of bytes_new: ", len(bytes_new))

del csvdata['Bytes']

csvdata['Bytes'] = bytes_new
csvdata

len of bytes_new:  153026


,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Flags,class,Bytes
0,0.000,TCP,26913_25,43618,EXT_SERVER,23.0,1,....S.,suspicious,46
1,0.000,TCP,EXT_SERVER,23,26913_25,43618.0,1,.A.R..,suspicious,40
2,0.000,TCP,26914_116,1817,EXT_SERVER,23.0,1,....S.,suspicious,46
3,0.000,TCP,EXT_SERVER,23,26914_116,1817.0,1,.A.R..,suspicious,40
4,0.001,TCP,26915_202,57272,EXT_SERVER,1433.0,1,....S.,suspicious,46
...,...,...,...,...,...,...,...,...,...,...
153021,67601.294,TCP,OPENSTACK_NET,49493,EXT_SERVER,8082.0,160681,.APRS.,normal,12200000
153022,67601.294,TCP,EXT_SERVER,8082,OPENSTACK_NET,49494.0,172810,.AP.S.,normal,465700000
153023,67601.294,TCP,OPENSTACK_NET,49494,EXT_SERVER,8082.0,170412,.APRS.,normal,28500000
153024,67601.103,TCP,EXT_SERVER,8082,OPENSTACK_NET,49495.0,148533,.AP.S.,normal,394900000


In [9]:
# encoder = ce.BinaryEncoder(cols=['Proto'])
# csvdata = encoder.fit_transform(csvdata)
# csvdata
csvdata = pd.get_dummies(csvdata, columns=['Proto'])


csvdata = pd.get_dummies(csvdata, columns=['Flags'])
# encoder = ce.BinaryEncoder(cols=['Flags'])
# csvdata = encoder.fit_transform(csvdata)
# csvdata


encoder = ce.BinaryEncoder(cols=['Src IP Addr'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Dst IP Addr'])
csvdata = encoder.fit_transform(csvdata)
csvdata


# encoder = ce.BinaryEncoder(cols=['Packets'])
# csvdata = encoder.fit_transform(csvdata)
# csvdata

encoder = ce.BinaryEncoder(cols=['Duration'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Src Pt'])
csvdata = encoder.fit_transform(csvdata)
csvdata

encoder = ce.BinaryEncoder(cols=['Dst Pt'])
csvdata = encoder.fit_transform(csvdata)
csvdata


,Duration_0,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Duration_6,Duration_7,Duration_8,Duration_9,Duration_10,Duration_11,Duration_12,Duration_13,Duration_14,Src IP Addr_0,Src IP Addr_1,Src IP Addr_2,Src IP Addr_3,Src IP Addr_4,Src IP Addr_5,Src IP Addr_6,Src IP Addr_7,Src IP Addr_8,Src IP Addr_9,Src IP Addr_10,Src IP Addr_11,Src IP Addr_12,Src IP Addr_13,Src Pt_0,Src Pt_1,Src Pt_2,Src Pt_3,Src Pt_4,Src Pt_5,Src Pt_6,Src Pt_7,Src Pt_8,Src Pt_9,Src Pt_10,...,Dst Pt_6,Dst Pt_7,Dst Pt_8,Dst Pt_9,Dst Pt_10,Dst Pt_11,Dst Pt_12,Dst Pt_13,Dst Pt_14,Dst Pt_15,Packets,class,Bytes,Proto_GRE,Proto_ICMP,Proto_TCP,Proto_UDP,Flags_ 0x52,Flags_ 0x5b,Flags_ 0xc2,Flags_ 0xd3,Flags_ 0xd6,Flags_ 0xd7,Flags_ 0xdb,Flags_ 0xdf,Flags_......,Flags_....S.,Flags_...R..,Flags_...RS.,Flags_.A....,Flags_.A..S.,Flags_.A..SF,Flags_.A.R..,Flags_.A.R.F,Flags_.A.RS.,Flags_.A.RSF,Flags_.AP.S.,Flags_.AP.SF,Flags_.APRS.,Flags_.APRSF
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,suspicious,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,suspicious,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,suspicious,46,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153021,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,...,1,1,1,0,1,1,1,0,1,0,160681,normal,12200000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153022,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,0,1,172810,normal,465700000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
153023,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,1,1,1,0,1,0,170412,normal,28500000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153024,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1,148533,normal,394900000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [10]:
for column in csvdata:
  if column.find('_') != -1:
    continue
  if column.find('class') != -1:
    continue
  print('-----')
  print(column)
  min = csvdata[column].min()
  max = csvdata[column].max()
  print(min)
  print(max)
  if min != 0:
    csvdata[column] = csvdata[column] - min
  if max != min:
    csvdata[column] = csvdata[column] / (max - min)

csvdata

-----
Packets
1
176609
-----
Bytes
28
509200000


,Duration_0,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Duration_6,Duration_7,Duration_8,Duration_9,Duration_10,Duration_11,Duration_12,Duration_13,Duration_14,Src IP Addr_0,Src IP Addr_1,Src IP Addr_2,Src IP Addr_3,Src IP Addr_4,Src IP Addr_5,Src IP Addr_6,Src IP Addr_7,Src IP Addr_8,Src IP Addr_9,Src IP Addr_10,Src IP Addr_11,Src IP Addr_12,Src IP Addr_13,Src Pt_0,Src Pt_1,Src Pt_2,Src Pt_3,Src Pt_4,Src Pt_5,Src Pt_6,Src Pt_7,Src Pt_8,Src Pt_9,Src Pt_10,...,Dst Pt_6,Dst Pt_7,Dst Pt_8,Dst Pt_9,Dst Pt_10,Dst Pt_11,Dst Pt_12,Dst Pt_13,Dst Pt_14,Dst Pt_15,Packets,class,Bytes,Proto_GRE,Proto_ICMP,Proto_TCP,Proto_UDP,Flags_ 0x52,Flags_ 0x5b,Flags_ 0xc2,Flags_ 0xd3,Flags_ 0xd6,Flags_ 0xd7,Flags_ 0xdb,Flags_ 0xdf,Flags_......,Flags_....S.,Flags_...R..,Flags_...RS.,Flags_.A....,Flags_.A..S.,Flags_.A..SF,Flags_.A.R..,Flags_.A.R.F,Flags_.A.RS.,Flags_.A.RSF,Flags_.AP.S.,Flags_.AP.SF,Flags_.APRS.,Flags_.APRSF
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0.000000,suspicious,2.356638e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0.000000,suspicious,2.356638e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0.000000,suspicious,3.534957e-08,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153021,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1,0,1,...,1,1,1,0,1,1,1,0,1,0,0.909812,normal,2.395910e-02,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153022,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,0,1,0.978489,normal,9.145719e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
153023,1,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,1,1,1,0,1,0,0.964911,normal,5.597010e-02,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
153024,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1,0.841026,normal,7.755302e-01,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [11]:
for column in csvdata:
  print(csvdata[column].value_counts())
  print()
  print()

0    152575
1       451
Name: Duration_0, dtype: int64


0    99037
1    53989
Name: Duration_1, dtype: int64


0    111863
1     41163
Name: Duration_2, dtype: int64


0    111358
1     41668
Name: Duration_3, dtype: int64


0    97378
1    55648
Name: Duration_4, dtype: int64


0    84536
1    68490
Name: Duration_5, dtype: int64


1    76962
0    76064
Name: Duration_6, dtype: int64


0    79714
1    73312
Name: Duration_7, dtype: int64


0    88779
1    64247
Name: Duration_8, dtype: int64


0    100231
1     52795
Name: Duration_9, dtype: int64


0    101291
1     51735
Name: Duration_10, dtype: int64


0    84255
1    68771
Name: Duration_11, dtype: int64


0    89880
1    63146
Name: Duration_12, dtype: int64


0    94904
1    58122
Name: Duration_13, dtype: int64


1    86619
0    66407
Name: Duration_14, dtype: int64


0    152757
1       269
Name: Src IP Addr_0, dtype: int64


0    114802
1     38224
Name: Src IP Addr_1, dtype: int64


0    119837
1     33189
Name: Src IP Add

In [12]:
csvdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153026 entries, 0 to 153025
Columns: 105 entries, Duration_0 to Flags_.APRSF
dtypes: float64(2), int64(75), object(1), uint8(27)
memory usage: 95.0+ MB


In [13]:

normal = csvdata.query('`class`=="normal"')
del normal['class']
print(len(normal))
# normal

6180


In [14]:

victim = csvdata.query('`class`=="victim"')
del victim['class']
print(len(victim))
# victim

5902


In [15]:

suspicious = csvdata.query('`class`=="suspicious"')
del suspicious['class']
print(len(suspicious))
# suspicious

97852


In [16]:
attacker = csvdata.query('`class`=="attacker"')
del attacker['class']
print(len(attacker))
# attacker

9255


In [17]:
unknown = csvdata.query('`class`=="unknown"')     ###############
del unknown['class']                        ###########
print(len(unknown))    

33837


In [18]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.e1 = nn.Linear(in_features=kwargs["input_shape"], out_features=165 ) # 1st Hidden Layer
        # self.e2 = nn.Linear(in_features=256, out_features=128)             
        # self.b  = nn.Linear(in_features=128, out_features=256)         
        self.output_layer = nn.Linear(in_features=165, out_features=kwargs["input_shape"])

        # self.e1 = nn.Linear(in_features=kwargs["input_shape"], out_features=512 ) # 1st Hidden Layer
        # self.e2 = nn.Linear(in_features=512, out_features=256)             
        # self.e3 = nn.Linear(in_features=256, out_features=128)             
        # self.b  = nn.Linear(in_features=128, out_features=256)         
        # self.o1  = nn.Linear(in_features=256, out_features=512)         
        # self.output_layer = nn.Linear(in_features=512, out_features=kwargs["input_shape"])

    def forward(self, features):
        out = torch.relu(self.e1(features))           #torch.relu(activation) #torch.RReLU(activation)
        # out = torch.relu(self.e2(out))                   
        # out = torch.relu(self.e3(out))                   

        # out = torch.relu(self.b(out))
        
        # out = torch.relu(self.o1(out))                   

        out = self.output_layer(out)
        out = torch.sigmoid(out)
        return out

    def _initialize_weights(self):
      #prints weights and bias of net
      #for item, param in net.state_dict().items():
        #print('item: ',item)

      for item, param in self.state_dict().items(): # either weight & bias set with gussian distribution

        #item is string
        #item.weight.data.normal_(mean=0.0, std=0.01)

        #below command is correct , up is false
        #self.state_dict()[item].data.normal_(mean=0.0, std=0.01)
        # nn.init.xavier_uniform_(item, gain=nn.init.calculate_gain('relu'))
        # nn.init.xavier_uniform_(item, gain=nn.init.calculate_gain('relu'))
        xavier(item.weight)
        xavier(item.bias)
      self.apply()


In [50]:
# size = 104
size = csvdata.shape[1] -1

In [51]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# size = 104
normal_classifier_model = AE(input_shape=size)
attacker_classifier_model = AE(input_shape=size)
victim_classifier_model = AE(input_shape=size)
unknown_classifier_model = AE(input_shape=size)
suspicious_classifier_model = AE(input_shape=size)

normal_classifier_model.to(device)
attacker_classifier_model.to(device)
victim_classifier_model.to(device)
unknown_classifier_model.to(device)
suspicious_classifier_model.to(device)


AE(
  (e1): Linear(in_features=104, out_features=165, bias=True)
  (output_layer): Linear(in_features=165, out_features=104, bias=True)
)

In [20]:
!cp /content/drive/"My Drive"/e3-multiclass/* ./

In [52]:
normal_classifier_model.load_state_dict(torch.load("10_cross_normal_fold.pth", map_location=torch.device('cpu')))
attacker_classifier_model.load_state_dict(torch.load("10_cross_attacker_fold.pth", map_location=torch.device('cpu')))
victim_classifier_model.load_state_dict(torch.load("10_cross_victim_fold.pth", map_location=torch.device('cpu')))
unknown_classifier_model.load_state_dict(torch.load("k_cross_unknown_AlwaysSaveType.pth", map_location=torch.device('cpu')))
suspicious_classifier_model.load_state_dict(torch.load("10_cross_suspicipus-104feature-165layer_fold.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [53]:
normal_classifier_model.eval()
attacker_classifier_model.eval()
victim_classifier_model.eval()
unknown_classifier_model.eval()
suspicious_classifier_model.eval()

AE(
  (e1): Linear(in_features=104, out_features=165, bias=True)
  (output_layer): Linear(in_features=165, out_features=104, bias=True)
)

In [23]:
def myloss2(input, target):
  A = input.detach().numpy()
  B = target.detach().numpy()
  C = np.absolute(A - B)
  return C.astype(np.float64)

In [24]:
# size = 104
size = csvdata.shape[1] -1

In [43]:
normal_threshold = np.zeros(size)
attacker_threshold = np.zeros(size)
victim_threshold = np.zeros(size)
unknown_threshold = np.zeros(size)
suspicious_threshold = np.zeros(size)

In [44]:
##calculate threshold of classes

for i in range(len(normal)):    ##########
    data = torch.from_numpy(np.array(normal.iloc[i])).float() ############
    data = data.to(device)
    sample = normal_classifier_model(data)
    
    loss2 = myloss2(data, sample)

    normal_threshold =  np.maximum(normal_threshold, loss2)

for i in range(len(attacker)):    ##########
    data = torch.from_numpy(np.array(attacker.iloc[i])).float() ############
    data = data.to(device)
    sample = attacker_classifier_model(data)   
    loss2 = myloss2(data, sample)
    attacker_threshold =  np.maximum(attacker_threshold, loss2)

for i in range(len(victim)):    ##########
    data = torch.from_numpy(np.array(victim.iloc[i])).float() ############
    data = data.to(device)
    sample = victim_classifier_model(data)    
    loss2 = myloss2(data, sample)
    victim_threshold =  np.maximum(victim_threshold, loss2)

for i in range(len(unknown)):    ##########
    data = torch.from_numpy(np.array(unknown.iloc[i])).float() ############
    data = data.to(device)
    sample = unknown_classifier_model(data)    
    loss2 = myloss2(data, sample)
    unknown_threshold =  np.maximum(unknown_threshold, loss2)


for i in range(len(suspicious)):    ##########
    data = torch.from_numpy(np.array(suspicious.iloc[i])).float() ############
    data = data.to(device)
    sample = suspicious_classifier_model(data)   
    loss2 = myloss2(data, sample)
    suspicious_threshold =  np.maximum(suspicious_threshold, loss2)
    

In [48]:
#binary classification

def binaryClassificationPredict(className, model, threshold):

  normal_counts = 0
  victim_counts = 0
  attacker_counts = 0
  suspicious_counts = 0
  unknown_counts = 0

  model.eval()

  for i in range(len(normal)):
      data = torch.from_numpy(np.array(normal.iloc[i])).float()
      data = data.to(device)
      sample = model(data)    
      loss2 = myloss2(data, sample)
      diff = threshold - loss2                           ##################################
      negative_count = np.sum(np.array(diff) < 0, axis=0)

      if negative_count == 0:
        normal_counts = normal_counts +1                      ##################################


  for i in range(len(victim)):
      data = torch.from_numpy(np.array(victim.iloc[i])).float()
      data = data.to(device)
      sample = model(data)    
      loss2 = myloss2(data, sample)
      diff = threshold - loss2                           ##################################
      negative_count = np.sum(np.array(diff) < 0, axis=0)

      if negative_count == 0:
        victim_counts = victim_counts +1                      ##################################


  for i in range(len(attacker)):
      data = torch.from_numpy(np.array(attacker.iloc[i])).float()
      data = data.to(device)
      sample = model(data)    
      loss2 = myloss2(data, sample)
      diff = threshold - loss2                           ##################################
      negative_count = np.sum(np.array(diff) < 0, axis=0)

      if negative_count == 0:
        attacker_counts = attacker_counts +1                      ##################################



  for i in range(len(suspicious)):
      data = torch.from_numpy(np.array(suspicious.iloc[i])).float()
      data = data.to(device)
      sample = model(data)    
      loss2 = myloss2(data, sample)
      diff = threshold - loss2                           ##################################
      negative_count = np.sum(np.array(diff) < 0, axis=0)

      if negative_count == 0:
        suspicious_counts = suspicious_counts +1                      ##################################


  for i in range(len(unknown)):
      data = torch.from_numpy(np.array(unknown.iloc[i])).float()
      data = data.to(device)
      sample = model(data)    
      loss2 = myloss2(data, sample)
      diff = threshold - loss2                           ##################################
      negative_count = np.sum(np.array(diff) < 0, axis=0)

      if negative_count == 0:
        unknown_counts = unknown_counts +1                      ##################################

  print(className, "binary classifier performance on dataset")

  print("Number of normal class data classifies as", className, ": ", normal_counts)
  print("Number of victim class data classifies as", className, ": ", victim_counts)
  print("Number of attacker class data classifies as", className, ": ", attacker_counts)
  print("Number of unknown class data classifies as", className, ": ", unknown_counts)
  print("Number of suspicious class data classifies as", className, ": ", suspicious_counts)
  print()

In [54]:
binaryClassificationPredict("Normal", normal_classifier_model, normal_threshold)
binaryClassificationPredict("Victim", victim_classifier_model, victim_threshold)
binaryClassificationPredict("Attacker", attacker_classifier_model, attacker_threshold)
binaryClassificationPredict("Unknown", unknown_classifier_model, unknown_threshold)
binaryClassificationPredict("Suspicious", suspicious_classifier_model, suspicious_threshold)

Normal  binary classifier performance on dataset
Number of normal class data classifies as Normal :  6180
Number of victim class data classifies as Normal :  0
Number of attacker class data classifies as Normal :  0
Number of unknown class data classifies as Normal :  0
Number of suspicious class data classifies as Normal :  0

Victim  binary classifier performance on dataset
Number of normal class data classifies as Victim :  0
Number of victim class data classifies as Victim :  5902
Number of attacker class data classifies as Victim :  0
Number of unknown class data classifies as Victim :  0
Number of suspicious class data classifies as Victim :  11

Attacker  binary classifier performance on dataset
Number of normal class data classifies as Attacker :  0
Number of victim class data classifies as Attacker :  0
Number of attacker class data classifies as Attacker :  9255
Number of unknown class data classifies as Attacker :  0
Number of suspicious class data classifies as Attacker :  

##Create Confusion matrix based on results:

<img src='https://drive.google.com/uc?id=1n6sw_zKQHYYWvEXI7gAyvCUW_HibojAR'>

<img src='https://drive.google.com/uc?id=1Z6UYQr9bn7jmrE0vnvCJknN8xgeVWbBW'>

<img src='https://drive.google.com/uc?id=17edFbOc06g4zk3-NKGp691msud7RAqKK'>

<img src='https://drive.google.com/uc?id=1GAumzPMqVfw1RgJLl0BGmAelumwG_QKY'>

<img src='https://drive.google.com/uc?id=1YO9HmZ8xRl-lYaJY3rm3CmyUN_3aDQLi'>



In [ ]:
def calculateEvaluationMetrics(className, tp, fp, fn, tn):
  accuracy = (tp + tn) / (tp + tn + fp + fn) * 100
  recall = tp / (tp + fn) * 100
  fpr = fp / (fp + tn) * 100
  precision = tp / (tp + fp) * 100
  f1 = 2 * (precision * recall) / (precision + recall) 

  print(className, "class results")
  print("Accuracy: ", accuracy)
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("FPR: ", fpr)
  print("f1-score: ", f1)
  print()

In [55]:
# calculateEvaluationMetrics("......", tp, fp, fn, tn)
calculateEvaluationMetrics("Normal", 6180, 0, 0, 146846)
calculateEvaluationMetrics("Attacker", 9255, 11, 0, 143760)
calculateEvaluationMetrics("Victim", 5902, 11, 0, 147113)
calculateEvaluationMetrics("Unknown", 33837, 149, 0, 119040)
calculateEvaluationMetrics("Suspicious", 97852, 47284, 0, 7890)

Normal class results
Accuracy:  100.0
Precision:  100.0
Recall:  100.0
FPR:  0.0
f1-score:  100.0

Attacker class results
Accuracy:  99.99281167906108
Precision:  99.8812864234837
Recall:  100.0
FPR:  0.007651056193529989
f1-score:  99.94060795853356

Victim class results
Accuracy:  99.99281167906108
Precision:  99.81396922036191
Recall:  100.0
FPR:  0.007476686332617384
f1-score:  99.90689801100295

Unknown class results
Accuracy:  99.90263092546364
Precision:  99.56158418172188
Recall:  100.0
FPR:  0.12501153629949074
f1-score:  99.78031051413237

Suspicious class results
Accuracy:  69.10067570216826
Precision:  67.420901774887
Recall:  100.0
FPR:  85.69978613114873
f1-score:  80.5406028281232



In [27]:
#multi classification

def multiClassificationPredict(mydata, className):
  normal_counts = 0
  victim_counts = 0
  attacker_counts = 0
  unknown_counts = 0
  suspicious_counts = 0


  for i in range(len(mydata)):
      data = torch.from_numpy(np.array(mydata.iloc[i])).float()
      data = data.to(device)

      dos_output = normal_classifier_model(data)    
      loss2 = myloss2(data, dos_output)
      diff = normal_threshold - loss2                           
      negative_count = np.sum(np.array(diff) < 0, axis=0)
      if negative_count == 0:
        normal_counts = normal_counts + 1
        continue                   


      output = victim_classifier_model(data)    
      loss2 = myloss2(data, output)
      diff = victim_threshold - loss2                           
      negative_count = np.sum(np.array(diff) < 0, axis=0)
      if negative_count == 0:
        victim_counts = victim_counts + 1
        continue



      output = attacker_classifier_model(data)    
      loss2 = myloss2(data, output)
      diff = attacker_threshold - loss2                           
      negative_count = np.sum(np.array(diff) < 0, axis=0)
      if negative_count == 0:
        attacker_counts = attacker_counts + 1
        continue 


      output = unknown_classifier_model(data)    
      loss2 = myloss2(data, output)
      diff = unknown_threshold - loss2                           
      negative_count = np.sum(np.array(diff) < 0, axis=0)
      if negative_count == 0:
        unknown_counts = unknown_counts + 1
        continue

      output = suspicious_classifier_model(data)    
      loss2 = myloss2(data, output)
      diff = suspicious_threshold - loss2                           
      negative_count = np.sum(np.array(diff) < 0, axis=0)
      if negative_count == 0:
        suspicious_counts = suspicious_counts + 1                          

  print(className," class data prediction:")
  print("Number of normal pridctions: ", normal_counts)
  print("Number of victim pridctions:", victim_counts)
  print("Number of attacker pridctions:", attacker_counts)
  print("Number of unknown pridctions:", unknown_counts)
  print("Number of suspicious pridctions:", suspicious_counts)
  print()

In [28]:
multiClassificationPredict(normal, "Normal")
multiClassificationPredict(victim, "Victim")
multiClassificationPredict(attacker, "Attacker")
multiClassificationPredict(unknown, "Unknown")
multiClassificationPredict(suspicious, "Suspicious")


Normal  class data prediction:
Number of normal pridctions:  6180
Number of victim pridctions: 0
Number of attacker pridctions: 0
Number of unknown pridctions: 0
Number of suspicious pridctions: 0

Victim  class data prediction:
Number of normal pridctions:  0
Number of victim pridctions: 5902
Number of attacker pridctions: 0
Number of unknown pridctions: 0
Number of suspicious pridctions: 0

Attacker  class data prediction:
Number of normal pridctions:  0
Number of victim pridctions: 0
Number of attacker pridctions: 9255
Number of unknown pridctions: 0
Number of suspicious pridctions: 0

Unknown  class data prediction:
Number of normal pridctions:  0
Number of victim pridctions: 0
Number of attacker pridctions: 0
Number of unknown pridctions: 33837
Number of suspicious pridctions: 0

Suspicious  class data prediction:
Number of normal pridctions:  0
Number of victim pridctions: 11
Number of attacker pridctions: 11
Number of unknown pridctions: 141
Number of suspicious pridctions: 9768

##Create Confusion matrix based on results
<img src='https://drive.google.com/uc?id=1fXbl4UPJENiVJfuzFUC68fSwQvh3SKY2'>

In [33]:
def calculateEvaluationMetrics(className, tp, fp, fn, tn):
  accuracy = (tp + tn) / (tp + tn + fp + fn) * 100
  recall = tp / (tp + fn) * 100
  fpr = fp / (fp + tn) * 100
  precision = tp / (tp + fp) * 100
  f1 = 2 * (precision * recall) / (precision + recall) 

  print(className, "class results")
  print("Accuracy: ", accuracy)
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("FPR: ", fpr)
  print("f1-score: ", f1)
  print()

In [35]:
# calculateEvaluationMetrics("......", tp, fp, fn, tn)
calculateEvaluationMetrics("Normal", 6180, 0, 0, 146846)
calculateEvaluationMetrics("Attacker", 9255, 11, 0, 143760)
calculateEvaluationMetrics("Victim", 5902, 11, 0, 147113)
calculateEvaluationMetrics("Unknown", 33837, 141, 0, 119048)
calculateEvaluationMetrics("Suspicious", 97689, 0, 11+11+141, 55174)

Normal class results
Accuracy:  100.0
Precision:  100.0
Recall:  100.0
FPR:  0.0
f1-score:  100.0

Attacker class results
Accuracy:  99.99281167906108
Precision:  99.8812864234837
Recall:  100.0
FPR:  0.007651056193529989
f1-score:  99.94060795853356

Victim class results
Accuracy:  99.99281167906108
Precision:  99.81396922036191
Recall:  100.0
FPR:  0.007476686332617384
f1-score:  99.90689801100295

Unknown class results
Accuracy:  99.90785879523742
Precision:  99.5850256048031
Recall:  100.0
FPR:  0.11829950750488719
f1-score:  99.79208139792082

Suspicious class results
Accuracy:  99.89348215335956
Precision:  100.0
Recall:  99.83342190246495
FPR:  0.0
f1-score:  99.9166415227497

